In [2]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import requests
from requests_html import HTML
import os
from urllib.parse import urlparse
import getpass

In [4]:

#esta funcion espera un determinado timeout (tiempo) y chequea si se cumplio una condicion(expected condicion), en este caso
#la condicion que chequea es que un determinado elemento en el buscador este visible, de esta forme se si la pagina ya cargo por completo
def pagina_cargando(driver,timeout=3, XPATH= None):
    try:
        element_present = EC.presence_of_element_located((By.XPATH, XPATH))
        WebDriverWait(driver,timeout).until(element_present)
    except:
        print("la pagina no cargo")
    
#ingresa a intragram con un determinado usuario y contraseña
#usa la funcion de espera para ver si carga la pagina y dsp usa el time.sleep para no hacer todo tan rapido y hacer pasar al programa mas como una persona
def ingresar(usuario=None,contra=None, headlees =False):
    if usuario == None:
        usuario = input("ingrese su usuario: ")
    if contra == None:
        contra = getpass.getpass("ingrese su contraseña: ")
    global browser
    if headlees:
        opcion = Options()
        opcion.add_argument("--headless")
        browser = webdriver.Chrome(options=opcion)
    else:
        browser = webdriver.Chrome()
    browser.get("https://www.instagram.com/")
    pagina_cargando(browser,timeout=2, XPATH= "//input[@name='username']")
    username_el = browser.find_element_by_name("username")
    username_el.send_keys(f"{usuario}")
    password_el = browser.find_element_by_name("password")
    password_el.send_keys(f"{contra}")
    time.sleep(1)
    password_el.submit()
    return browser
        

#toma todos los enlaces de todas las fotos de la pagina principal del nombre buscado
def enlaces_de_fotos(browser, nombre, cantidad_de_fotos):
    URL = f"https://www.instagram.com/{nombre}/"
    browser.get(URL)
    lista_de_enlace= []
    Xpath = "//a[contains(@href, '/p/')]"
    pagina_cargando(browser, timeout=4, XPATH=Xpath)
    links = browser.find_elements_by_xpath(Xpath)
    for i in range(cantidad_de_fotos):
        lista_de_enlace.append(links[i].get_attribute("href"))
    return lista_de_enlace
    
    
#de un conjunto de elementos fotos, toma solamente el mas grande de estos, esto es ya que al ser muy dificl diferenciar las distintas fotos
#que hay en un pagina de inta(hay un monton, por ej los circulitos de las parsonas que comentan la foto principal)
#eligo la mas grande que siempre va a ser la principal
def encontrar_img_max(fotos):
    tamaño = 0
    indice = 0
    for j,f in enumerate(fotos):
        sizes = f.get_attribute("sizes").split("p")
        try:
            size = int(sizes[0])
            if size >= tamaño:
                tamaño = size
                indice = j
        except:
            pass
    return fotos[indice].get_attribute("src")

#crea una carpeta con el nombre de la persona a la que se le descargan las fotos
def crear_carpeta(nombre):
    base_dir = os.getcwd()
    main_dir = os.path.join(base_dir, "instagram Fotos")
    global fotos_dir
    fotos_dir = os.path.join(main_dir,f"{nombre}")
    os.makedirs(main_dir, exist_ok=True)
    os.makedirs(fotos_dir, exist_ok=True)
    return


#toma los link de todas las fotos principales
def descargar_fotos(browser,nombre, n=3):
    crear_carpeta(nombre=nombre)
    links = enlaces_de_fotos(browser=browser, nombre =nombre, cantidad_de_fotos= n)
    for link in links:
        browser.get(link)
        time.sleep(3)
        fotos = browser.find_elements_by_xpath("//img")
        img_link = encontrar_img_max(fotos)
        time.sleep(3)
        guardar_foto(img_link)
    browser.Close()
        

#primero chequea que el archivo que voy a guardar no exista asi no lo guarda dos veces, despues se hace la request y se guarda la foto
def guardar_foto(img_link):
    base_url = urlparse(img_link).path
    file = os.path.dirname(base_url)
    file_name = base_url.strip(file)
    file_path = os.path.join(fotos_dir,file_name)
    if os.path.exists(file_path):
        return
    with requests.get(img_link) as r:
        r.raise_for_status()
        with open(file_path, "wb") as f:
            for chunk in r.iter_content():
                f.write(chunk)
    return
    